In [3]:
import pandas as pd
import numpy as np
import pickle

In [5]:
tracks = pickle.load(open('tracks_df.pkl', 'rb'))

In [6]:
hits = pickle.load(open('hits_df.pkl', 'rb'))

In [11]:
data = pd.concat([tracks, hits], axis=0).reset_index(drop=True)